Loading Dataset

In [12]:
import pandas as pd

df = pd.read_csv('xor.csv')

In [13]:
X = df[["x1", "x2"]].values
y = df["class label"].values

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

In [22]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class PytorchMLP(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.linear_1 = torch.nn.Linear(num_features, 25)    
        self.linear_2 = torch.nn.Linear(25, 15)
        self.linear_3 = torch.nn.Linear(15, num_classes)
    
    def forward(self, x):
        x = self.linear_1(x)
        x = F.relu(x)
        x= self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)
        return x

In [23]:
from torch.utils.data import DataLoader, Dataset
class XORData(Dataset):
    def __init__(self, features, labels):
        self.features= torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    

train_dataset = XORData(X_train, y_train)
val_dataset = XORData(X_val, y_val)
test_dataset = XORData(X_test, y_test)


In [24]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validatation_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [25]:

def compute_accuracy(model, dataloader):

    model = model.eval()
    
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        
        with torch.inference_mode(): # basically the same as torch.no_grad
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)

        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

In [27]:
import torch.optim as optim
import torch.nn.functional as F 

torch.manual_seed(1)
#import SGD optimizer
model = PytorchMLP(2, 2)
optimizer = optim.SGD(model.parameters(), lr=0.01)
epochs= 10


for epoch in range(epochs): 
    model= model.train()
    
    for batch_idx, (features ,labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch: {epoch}, Batch index: {batch_idx}, Loss: {loss.item()}")
    train_accuracy = compute_accuracy(model, train_loader)
    val_accuracy = compute_accuracy(model, validatation_loader)
    print(f"Epoch: {epoch}, Train accuracy: {train_accuracy}, Validation accuracy: {val_accuracy}")
    
    

Epoch: 0, Batch index: 0, Loss: 0.675772488117218
Epoch: 0, Batch index: 1, Loss: 0.6912350654602051
Epoch: 0, Batch index: 2, Loss: 0.6793786883354187
Epoch: 0, Batch index: 3, Loss: 0.6760478615760803
Epoch: 0, Batch index: 4, Loss: 0.6922560930252075
Epoch: 0, Batch index: 5, Loss: 0.6893620491027832
Epoch: 0, Batch index: 6, Loss: 0.6598774790763855
Epoch: 0, Batch index: 7, Loss: 0.6838384866714478
Epoch: 0, Batch index: 8, Loss: 0.6826114654541016
Epoch: 0, Batch index: 9, Loss: 0.690500795841217
Epoch: 0, Batch index: 10, Loss: 0.6667635440826416
Epoch: 0, Batch index: 11, Loss: 0.659843385219574
Epoch: 0, Batch index: 12, Loss: 0.682585597038269
Epoch: 0, Batch index: 13, Loss: 0.6836134195327759
Epoch: 0, Batch index: 14, Loss: 0.659523069858551
Epoch: 0, Batch index: 15, Loss: 0.6781588792800903
Epoch: 0, Batch index: 16, Loss: 0.6774669289588928
Epoch: 0, Batch index: 17, Loss: 0.6672713160514832
Epoch: 0, Train accuracy: 0.5794066190719604, Validation accuracy: 0.5625
Epoch

In [28]:
train_accuracy = compute_accuracy(model, train_loader)
val_accuracy = compute_accuracy(model, validatation_loader)
test_accuracy = compute_accuracy(model, test_loader)

print(f"Train accuracy: {train_accuracy}, Validation accuracy: {val_accuracy}, Test accuracy: {test_accuracy}")

Train accuracy: 0.8289703130722046, Validation accuracy: 0.75, Test accuracy: 0.7876105904579163
